In [1]:
flex_title = "Analyse nutritionnelle des produits français"
flex_subtitle = "Données Open Food Facts"
flex_include_source = True
flex_orientation = "rows"

In [2]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets

In [3]:
data = pd.read_csv("data/fr.openfoodfacts.org.products.cleaned.csv", low_memory=False)

c:\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (0,3,5,26,27,28,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Sidebar

### Filtre

In [4]:
components = []

nutri = ["Tous les grades", "a", "b", "c", "d", "e"]

brands = [
     "Tous les distributeurs",
     "carrefour",
     "auchan",
     "u",
     "leader-price",
     "casino",
     "cora",
     "picard",
     "monoprix",
     "fleury-michon",
     "nestle"]

nutri_label = widgets.Label(value="Nutri-Score :")
nutri_score = widgets.Dropdown(options=nutri, value="a")
components.extend([nutri_label, nutri_score])


brands_label = widgets.Label(value="Distributeur :")
brands_score = widgets.Dropdown(options=brands, value="Tous les distributeurs")
components.extend([brands_label, brands_score])


products_text = widgets.Textarea(value='Tous les produits',
                                 placeholder='Ecrire quelque chose',
                                 description='Produit :',
                                 disabled=False
                                 )

display(widgets.VBox([nutri_label, nutri_score, brands_label,
        brands_score, products_text]))

### Variables et Widgets

In [5]:
components = []

axis_map_x = {
    "nutrition-score-fr_100g": "nutrition-score-fr_100g",   
    "proteins_100g": "proteins_100g",
    "sugars_100g": "sugars_100g",
    "fat_100g": "fat_100g",
    "saturated-fat_100g": "saturated-fat_100g",
    "vitamin-d_100g":"vitamin-d_100g",
    "vitamin-c_100g":"vitamin-c_100g",  
    "sodium_100g":"sodium_100g",
    "potassium_100g":"potassium_100g",
    "calcium_100g":"calcium_100g",
    "magnesium_100g":"magnesium_100g",
    "iron_100g":"iron_100g",
    "zinc_100g":"zinc_100g",
}
    
axis_map_y = {
    "nutrition-score-fr_100g": "nutrition-score-fr_100g",   
    "proteins_100g": "proteins_100g",
    "sugars_100g": "sugars_100g",
    "fat_100g": "fat_100g",
    "saturated-fat_100g": "saturated-fat_100g",
    "vitamin-d_100g":"vitamin-d_100g",
    "vitamin-c_100g":"vitamin-c_100g",  
    "sodium_100g":"sodium_100g",
    "potassium_100g":"potassium_100g",
    "calcium_100g":"calcium_100g",
    "magnesium_100g":"magnesium_100g",
    "iron_100g":"iron_100g",
    "zinc_100g":"zinc_100g",
}
    
x_axis_label = widgets.Label(value="Variable horizontale X:")
x_axis = widgets.Dropdown(options=list(axis_map_x.items()), value="sugars_100g")
components.extend([x_axis_label, x_axis])

y_axis_label = widgets.Label(value="Variable verticale Y:")
y_axis = widgets.Dropdown(options=list(axis_map_y.items()), value="proteins_100g")
components.extend([y_axis_label, y_axis])

note = "Note: Les scores nutrition fr s'étendent de -15 à +40."
note_label = widgets.Label(value=note)
components.append(note_label)

display(widgets.VBox([x_axis_label, x_axis, y_axis_label, y_axis]))

# Graphique

### Nuage de points

In [6]:
out = widgets.Output()

In [7]:
cols = ['code','url','product_name','brands_tags','main_category_fr','nutrition_grade_fr',
        'nutrition-score-fr_100g','nutrition-score-uk_100g','proteins_100g', 'sugars_100g', 
        'fat_100g', 'saturated-fat_100g','omega-3-fat_100g','cholesterol_100g', 'fiber_100g',
        'vitamin-a_100g','vitamin-d_100g','vitamin-c_100g','sodium_100g',
        'potassium_100g', 'calcium_100g','magnesium_100g', 'iron_100g', 'zinc_100g','caffeine_100g'] 

data_df = pd.DataFrame(data, columns=cols)

fig = go.Figure()
plot = go.Scatter(x=[], y=[], mode="markers", text=[], hoverinfo="text", marker=dict(color=[], size=7))
fig.add_trace(plot)

margin = go.layout.Margin(l=20, r=20, b=20, t=30)
fig = fig.update_layout(margin=margin)

In [8]:
def select_products():
    selected = data_df
    if (nutri_score.value != "Tous les grades"):
        selected = selected[selected.nutrition_grade_fr.str.contains(nutri_score.value)==True]
    if (brands_score.value != "Tous les distributeurs"):
        selected = selected[selected.brands_tags.str.contains(brands_score.value)==True]
    if (products_text.value != "Tous les produits"):
        selected = selected[selected.product_name.str.contains(products_text.value)==True]
    return selected


def on_value_change(change):
    with out:
        
        df = select_products()
        x_name = x_axis.value
        y_name = y_axis.value

        fig.data[0]['x'] = df[x_name]
        fig.data[0]['y'] = df[y_name]

        if (nutri_score.value == "a"):
            fig.data[0]['marker']['color'] = "#038141"
        if (nutri_score.value == "b"):
            fig.data[0]['marker']['color'] = "#85bb2f"
        if (nutri_score.value == "c"):
            fig.data[0]['marker']['color'] = "#fecb02"
        if (nutri_score.value == "d"):
            fig.data[0]['marker']['color'] = "#ee8100"
        if (nutri_score.value == "e"):
            fig.data[0]['marker']['color'] = "#e63e11"
        if (nutri_score.value == "All"):
            fig.data[0]['marker']['color'] = "purple"
    
        
        fig.data[0]['text'] = df["product_name"] + "<br>" + "nutri score :" + df["nutrition_grade_fr"].astype(str) + "<br>" + "marque :" + df["brands_tags"].astype(str)
    
        fig.update_xaxes(title_text=x_axis.label)
        fig.update_yaxes(title_text=y_axis.label)
        fig.update_layout(title="%d produits selectionnés" % len(df))
        
        clear_output(wait=True)
        fig.show()
        

nutri_score.observe(on_value_change, names="value")
x_axis.observe(on_value_change, names="value")
y_axis.observe(on_value_change, names="value") 
brands_score.observe(on_value_change, names="value")
products_text.observe(on_value_change, names="value")

In [9]:
on_value_change(None)
out

Output()